In [4]:
import wandb
wandb.login(key="c26df6b59bfb128917e73bbb00a79ca7e9324a11")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
import logging
import shutil
import os

logging.basicConfig(level=logging.INFO)

train_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Train.xlsx"
test_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Test.xlsx"
train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data[['q_body', 'category', 'severity']]

valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]
valid_severity = ["حرج", "غير حرج"]

all_data = all_data[all_data["category"].isin(valid_categories) & all_data["severity"].isin(valid_severity)]
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)

train_df, test_df = train_test_split(all_data, test_size=0.2, random_state=42, stratify=all_data["severity"])

severity_mapping = {sev: i for i, sev in enumerate(valid_severity)}
train_df['label'] = train_df['severity'].map(severity_mapping)
test_df['label'] = test_df['severity'].map(severity_mapping)

model_name = "distilbert/distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["q_body"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_pandas(train_df[['q_body', 'label']])
test_dataset = Dataset.from_pandas(test_df[['q_body', 'label']])
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

base_model = AutoModel.from_pretrained(model_name)

class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = self.loss_fn(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits}

model = CustomModel(base_model, num_labels=len(valid_severity))

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=3e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("\n🔹 Starting Training...")
trainer.train()

test_metrics = trainer.evaluate(test_dataset)
print("\n🔹 Test Metrics:", test_metrics)

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

print("\n🔹 Confusion Matrix:")
print(confusion_matrix(labels, preds))
print("\n🔹 Classification Report:")
print(classification_report(labels, preds))

def save_complete_model(model, tokenizer, severity_mapping, save_path):
    model.base_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    torch.save({
        'classifier_state': model.classifier.state_dict(),
        'num_labels': model.classifier.out_features
    }, f"{save_path}/classifier_state.pt")
    with open(f"{save_path}/severity_mapping.pkl", "wb") as f:
        pickle.dump(severity_mapping, f)

save_complete_model(trainer.model, tokenizer, severity_mapping, "DistilBERT-Severity")

def load_complete_model(model_path):
    base_model = AutoModel.from_pretrained(model_path)
    classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    return model

tokenizer = AutoTokenizer.from_pretrained("DistilBERT-Severity")
model2 = load_complete_model("DistilBERT-Severity")
print("✅ Model Loaded Successfully!")

def predict_severity(text, model, tokenizer, severity_mapping):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs["logits"]
    predicted_label = torch.argmax(logits, dim=-1).item()
    severity_mapping_reverse = {v: k for k, v in severity_mapping.items()}
    return severity_mapping_reverse[predicted_label]

test_samples = ["الاعراض شديده", "تحتاج الى راحه فقط", "حاله متوسطه", "نزيف حاد"]
for text in test_samples:
    predicted_severity = predict_severity(text, model2, tokenizer, severity_mapping)
    print(f"\n🔹 Input: {text}")
    print(f"Predicted Severity: {predicted_severity}")

shutil.make_archive("DistilBERT-Severity", 'zip', "DistilBERT-Severity")
print("\n✅ Model Saved & Zipped for Download!")

from IPython.display import FileLink
FileLink(r'DistilBERT-Severity.zip')


Map:   0%|          | 0/159143 [00:00<?, ? examples/s]

Map:   0%|          | 0/39786 [00:00<?, ? examples/s]

/tmp/ipykernel_35/529349445.py:111: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🔹 Starting Training...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t


🔹 Test Metrics: {'eval_loss': 0.33960527181625366, 'eval_accuracy': 0.8551500527823858, 'eval_precision': 0.8580098920731865, 'eval_recall': 0.8551500527823858, 'eval_f1': 0.8564093127047725, 'eval_runtime': 89.3446, 'eval_samples_per_second': 445.31, 'eval_steps_per_second': 2.328, 'epoch': 10.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



🔹 Confusion Matrix:
[[26933  3142]
 [ 2621  7090]]

🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.90     30075
           1       0.69      0.73      0.71      9711

    accuracy                           0.86     39786
   macro avg       0.80      0.81      0.81     39786
weighted avg       0.86      0.86      0.86     39786

✅ Model Loaded Successfully!

🔹 Input: الاعراض شديده
Predicted Severity: حرج

🔹 Input: تحتاج الى راحه فقط
Predicted Severity: حرج

🔹 Input: حاله متوسطه
Predicted Severity: حرج

🔹 Input: نزيف حاد
Predicted Severity: حرج

✅ Model Saved & Zipped for Download!


/kaggle/working/DistilBERT-Severity.zip